# Selección de características con sklearn

En esta primera iteración de selección de características utilizando SelectPercentile con la prueba chi2 para reducir al 50% de las catacterísticas iniciales se tiene que:
- las variables principales para estimar el comportamiento de pago de créditos regulares son las variables numéricas "monto" y "cuotas" (plazo de crédito), y las variables categóricas "pagaduria", "estado_laboral" del asociado, y "destinacion" del crédito.
- las variables principales para estimar el comportamiento de pago de créditos rotativos se reducen a las variables numéricas "monto" y "cuotas" (plazo de crédito); presumiendo que esto puede ocurrir dadas las características puntuales que describen los créditos rotativos y la mayor limitante de información para estos créditos.

In [1]:
# imports

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2

Lectura de datos:

In [2]:
# directorios
DATA_DIR = Path("../data")

# datos
DATA_PROCESSED_FILE = DATA_DIR / "interno-procesados.parquet"

In [3]:
df = pd.read_parquet(DATA_PROCESSED_FILE)
df.shape

(889, 15)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   categoria_cod       889 non-null    object        
 1   cuota               889 non-null    float64       
 2   destinacion_cod     889 non-null    object        
 3   emision             889 non-null    datetime64[ns]
 4   estado_cliente_cod  824 non-null    object        
 5   estado_laboral_cod  824 non-null    object        
 6   garantia_cod        889 non-null    object        
 7   monto               889 non-null    float64       
 8   ncuotas             889 non-null    int64         
 9   oficina_cod         824 non-null    object        
 10  pagaduria_cod       889 non-null    object        
 11  tasa                889 non-null    float64       
 12  tipo                889 non-null    object        
 13  tipo_pago_cod       889 non-null    object        

Definición de variables:

In [5]:
categorical = [
    "destinacion_cod",
    "garantia_cod",
    "pagaduria_cod",
    "tipo",
    "tipo_pago_cod"
]
categorical_regular = ["estado_cliente_cod", "estado_laboral_cod", "oficina_cod"]
numerical = ["cuota", "monto", "ncuotas", "tasa"]
target = "categoria_cod"

Definición de procesamiento:

In [6]:
encoder_categorical = make_column_transformer(
    (OneHotEncoder(), categorical),
    remainder="passthrough"
)
encoder_categorical_regular = make_column_transformer(
    (OneHotEncoder(), categorical +  categorical_regular),
    remainder="passthrough"
)

encoder_pipe = make_column_transformer(
    (OneHotEncoder(), categorical),
    remainder='passthrough',
)
encoder_pipe_regular = make_column_transformer(
    (OneHotEncoder(), categorical + categorical_regular),
    remainder='passthrough',
)
pipe = make_pipeline(encoder_pipe, SelectPercentile(chi2, percentile=50))
pipe_regular = make_pipeline(encoder_pipe_regular, SelectPercentile(chi2, percentile=50))

In [7]:
# funciones

def get_X_y(tipo = None):
    if tipo is None:
        X = df[categorical + numerical]
        y = df[target]
    elif tipo == "regular":
        data = df.query("tipo == 'regular'")
        X = data[categorical + categorical_regular + numerical]
        y = data[target]
    elif tipo == "rotativo":
        data = df.query("tipo == 'rotativo'")
        X = data[categorical + numerical]
        y = data[target]
    else:
        raise Exception("invalid tipo")

    return (X, y)


def get_selected_features(pipe):
    encoder = pipe.named_steps["columntransformer"]
    encoded_feature_names = encoder.get_feature_names_out()
    
    select = pipe.named_steps["selectpercentile"]
    scores = select.scores_
    
    feature_scores = pd.DataFrame({
        'feature': encoded_feature_names,
        'score': scores
    })
    
    idx_selected_features = select.get_support()
    selected_features = feature_scores[idx_selected_features]
    
    return (
        selected_features
        .sort_values(by="score", ascending=False)
        .reset_index(drop=True)
    )

La selección de características considerando créditos rotativos muestra que las variables de monto y número de cuotas tienen el mayor peso predictivo sobre el comportamiento de pago del asociado y, como se esperaría, los valores con mayor frecuencia en variables categóricas tienen un mayor peso:

In [8]:
X, y = get_X_y("rotativo")

X_encoded = encoder_categorical.fit_transform(X, y)
nfeatures = X_encoded.shape[1] if isinstance(X_encoded, np.ndarray) else X_encoded.toarray().shape[1]
print(f"características codificadas:\tnfeatures={nfeatures}")

X_featured = pipe.fit_transform(X, y)
features = get_selected_features(pipe)
nfeatures = features.shape[0]
print(f"características seleccionadas:\tnfeatures={nfeatures}")

features

características codificadas:	nfeatures=10
características seleccionadas:	nfeatures=5


,feature,score
0,remainder__cuota,598.638892
1,remainder__monto,49.728205
2,onehotencoder__garantia_cod_C5,2.276786
3,onehotencoder__garantia_cod_C3,1.011905
4,remainder__tasa,0.032659


Características codificadas y seleccionadas automáticamente para créditos regulares:

In [9]:
X, y = get_X_y("regular")

X_encoded = encoder_categorical_regular.fit_transform(X, y)
nfeatures = X_encoded.shape[1] if isinstance(X_encoded, np.ndarray) else X_encoded.toarray().shape[1]
print(f"características codificadas:\tnfeatures={nfeatures}")

X_featured = pipe_regular.fit_transform(X, y)
features = get_selected_features(pipe_regular)
nfeatures = features.shape[0]
print(f"características seleccionadas:\tnfeatures={nfeatures}")

features

características codificadas:	nfeatures=57
características seleccionadas:	nfeatures=28


,feature,score
0,remainder__monto,1882.658137
1,remainder__ncuotas,1244.128130
2,onehotencoder__tipo_pago_cod_C1,214.065358
3,onehotencoder__tipo_pago_cod_C2,142.999126
4,onehotencoder__pagaduria_cod_C20,141.692034
5,onehotencoder__pagaduria_cod_C05,136.637449
6,onehotencoder__estado_laboral_cod_C3,118.726261
7,onehotencoder__estado_cliente_cod_C2,107.275220
8,onehotencoder__destinacion_cod_C03,74.381815
9,onehotencoder__estado_laboral_cod_C6,66.097977


Características para créditos regulares y créditos rotativos:

In [10]:
X, y = get_X_y()

X_encoded = encoder_categorical.fit_transform(X, y)
nfeatures = X_encoded.shape[1] if isinstance(X_encoded, np.ndarray) else X_encoded.toarray().shape[1]
print(f"características codificadas:\tnfeatures={nfeatures}")

X_featured = pipe.fit_transform(X, y)
features = get_selected_features(pipe)
nfeatures = features.shape[0]
print(f"características seleccionadas:\tnfeatures={nfeatures}")

features

características codificadas:	nfeatures=45
características seleccionadas:	nfeatures=22


,feature,score
0,remainder__monto,1806.669123
1,remainder__ncuotas,1267.877032
2,onehotencoder__tipo_pago_cod_C1,235.469296
3,onehotencoder__pagaduria_cod_C05,147.293127
4,onehotencoder__tipo_pago_cod_C2,139.006919
5,onehotencoder__pagaduria_cod_C20,137.777422
6,onehotencoder__destinacion_cod_C03,78.452251
7,onehotencoder__destinacion_cod_C01,46.696800
8,onehotencoder__destinacion_cod_C18,36.975879
9,onehotencoder__tipo_rotativo,34.713750
